In [1]:
%%capture
!pip install chromadb
!pip install requests

In [2]:
import os
import chromadb
import pandas as pd
from google.colab import drive
from chromadb import Documents, EmbeddingFunction, Embeddings
import chromadb.utils.embedding_functions as embedding_functions

from chromadb import Documents, EmbeddingFunction, Embeddings

from transformers import AutoConfig, AutoTokenizer, AutoModel
import torch

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from drive.MyDrive.nlp.tools import extraction,prompt

In [5]:
df = pd.read_csv('./drive/MyDrive/nlp/final_dataset.csv')

In [6]:
chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/nlp")


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:

config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("drippypale/parsbert-movie-finetuned")
model = AutoModel.from_pretrained("drippypale/parsbert-movie-finetuned").to(device)

In [8]:
class MyEmbeddingParsBert(EmbeddingFunction):
    def __init__(self, model,tokenizer):
      self.model = model
      self.tokenizer =  tokenizer
    def get_embedding(self,text):
      inputs = self.tokenizer(text, return_tensors='pt',padding=True, truncation=True, max_length = 512).to(device)
      outputs = self.model(**inputs, output_hidden_states=True)
      embedding = outputs.hidden_states[-1][0,0,:]
      return embedding.tolist()

    def __call__(self, input: Documents) -> Embeddings:
        embeddings = [self.get_embedding(doc) for doc in input]
        return embeddings

In [9]:

embedding_function = MyEmbeddingParsBert(model,tokenizer)

In [10]:
collection = chroma_client.get_or_create_collection(name = "bertpersion",embedding_function=embedding_function)


In [ ]:
totall = 0

for index, row in df.iterrows():
    if totall %300 == 0:
      print('item :',totall)


    ids = [extraction.extrac_id(row)]
    text = [prompt.create_promt_embeding(row)]
    metadatas = [extraction.meta_data(row)]
    collection.add(
    documents=text,
    metadatas=metadatas,
    ids=ids
    )
    totall += 1